<a href="https://colab.research.google.com/github/sirakik/japan_graph/blob/master/%E8%88%AA%E7%A9%BA%E3%83%87%E3%83%BC%E3%82%BF%E6%95%B4%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import xml.etree.ElementTree as ET

国土交通省　国土数値情報｜空港間流通量　https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-S10b-v1_1.html  
平成26年度データ

### 手作業での変更
国際便のデータを削除  
空港座標を削除
linkを削除(ksj, gml)

###各都道府県の空港データ

In [ ]:
airport={0:['稚内', '利尻', '旭川', '紋別', '女満別', '中標津', '釧路', '帯広', '丘珠', '函館', '新千歳', '奥尻'], 1:['青森', '三沢'], 2:['花巻'], 3:['仙台'], 4:['秋田', '大館'], 5:['庄内', '山形'], 6:['福島'], 7:['茨城'], 8:[], 9:[], 10:[], 11:['成田'], 12:['東京', '調布', '大島', '新島', '神津島', '三宅島', '八丈島', '利島', '御蔵島', '青ヶ島'], 13:[], 14:['新潟'], 15:['富山'], 16:['小松', '能登', '金沢'], 17:[], 18:[], 19:['松本'], 20:[], 21:['静岡'], 22:['名古屋', '中部'], 23:[], 24:[], 25:[], 26:['大阪', '関西'], 27:['神戸', '但馬'], 28:[], 29:['南紀白浜'], 30:['鳥取', '米子'], 31:['出雲', '隠岐', '石見'], 32:['岡山'], 33:['広島'], 34:['山口宇部', '岩国'], 35:['徳島'], 36:['高松'], 37:['松山'], 38:['高知'], 39:['北九州', '福岡'], 40:['佐賀', '対馬', '壱岐', '福江'], 41:['長崎'], 42:['熊本', '天草'], 43:['大分'], 44:['宮崎'], 45:['鹿児島', '種子島', '屋久島', '奄美', '喜界島', '徳之島', '沖永良部', '与論'], 46:['那覇', '粟国', '久米島', '北大東島', '南大東島', '宮古', '下地島', '多良間', '石垣', '与那国']}

### xml　使ってみた

In [ ]:
# 白木のgithubからダウンロード
!wget -q https://raw.githubusercontent.com/sirakik/japan_graph/master/S10b-14.xml -O airport.xml

airport_xml = ET.parse('airport.xml')
root = airport_xml.getroot()

In [ ]:
def find_key(value):
    for k, v in airport.items():
        if value in v:
          return k

pas_matrix = np.zeros((47, 47)) # 累計旅客数カウント用
for n in range(0, len(root), 2): # 東京->新千歳, 新千歳->東京のようにひっくり返したものが連続するので1つ飛ばしする
    dep = root[n].find('depAport').text # 出発空港
    arr = root[n].find('arrAport').text # 到着空港
    frequency = int(root[n].find('distAirRt').text) #頻度 週当たりの本数
    try:
      pas = int(root[n].find('numOfPass').text) # 年間旅客数
    except:
      continue
    
    # 空港を都道府県番号に変換
    dep_key = find_key(dep)
    arr_key = find_key(arr)
    
    if dep_key == arr_key: # 県内移動は無視
        continue
    
    if dep_key > arr_key:
        x = arr_key
        y = dep_key
    else:
        x = dep_key
        y = arr_key
    
    # 旅客数を加算
    pas_matrix[x][y] += pas

In [ ]:
print(np.max(pas_matrix))

6184077.0


In [ ]:
# 旅客数の多さによってグラフを構築

S = 100000 
# 0<旅客数<=S　を　few_pas_graph
# S<旅客数 を many_pas_graph

few_pas_graph = []
many_pas_graph = []
for i, n in enumerate(pas_matrix.flatten()):
    x = i // 47
    y = i % 47
    if (0 < n) & (n <= S):
        few_pas_graph.append((x, y))
    elif (S < n):
        many_pas_graph.append((x, y))
    else:
      pass

print(len(few_pas_graph))
print(len(many_pas_graph))

print(few_pas_graph)
print(many_pas_graph)

74
56
[(0, 1), (0, 2), (0, 4), (0, 6), (0, 7), (0, 14), (0, 15), (0, 16), (0, 19), (0, 21), (0, 32), (0, 33), (0, 46), (1, 22), (1, 26), (2, 22), (2, 26), (2, 39), (3, 11), (3, 16), (3, 33), (3, 46), (4, 22), (4, 26), (5, 26), (6, 26), (7, 27), (7, 30), (7, 46), (11, 14), (11, 16), (11, 30), (11, 33), (11, 36), (11, 37), (11, 43), (11, 45), (12, 22), (12, 29), (14, 22), (14, 39), (14, 46), (16, 39), (16, 46), (19, 39), (21, 39), (21, 45), (21, 46), (22, 37), (22, 38), (22, 41), (22, 43), (22, 44), (26, 27), (26, 31), (26, 43), (27, 30), (27, 41), (27, 45), (31, 39), (32, 46), (33, 46), (35, 39), (36, 46), (37, 39), (37, 45), (37, 46), (38, 39), (39, 42), (39, 45), (40, 41), (41, 46), (42, 46), (44, 46)]
[(0, 3), (0, 11), (0, 12), (0, 22), (0, 26), (0, 27), (0, 39), (1, 12), (3, 22), (3, 26), (3, 39), (4, 12), (5, 12), (11, 22), (11, 26), (11, 39), (11, 46), (12, 15), (12, 16), (12, 26), (12, 27), (12, 30), (12, 31), (12, 32), (12, 33), (12, 34), (12, 35), (12, 36), (12, 37), (12, 38), 

完成した，接続関係はコピペで移動してます